## Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

## Reading and Processing the Files

In [2]:
filename=input("Enter File Name:-")
df=pd.read_excel(filename+'.xlsx')

Enter File Name:-Data


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
df=df[['Request Number','Request Owner','Account','Local Category','Priority','Client Alias','# of Assets Impacted','Hours to First Response','Country Routing','Contact','Sub Type 3','Subject','Work Start Date','Target Date','Work End Date','Date/Time Closed','Submitted Date/Time','Private Notes','Client Initiated Date/Time','Solution Code','Solution']]
df=df.set_axis(['req_num','req_owner','account','local_cat','priority','client_alias','assets_impacted','hours_to_response','country_routing','contact','sub_type_3','subject','start_date','target_date','end_date','dtclose','submitted_date','private_notes','client_initiated_date','solution_code','solution'],axis='columns')
df=df[['req_num','req_owner','account','local_cat','priority','client_alias','assets_impacted','hours_to_response','country_routing','sub_type_3','subject','start_date','target_date','end_date','dtclose','submitted_date','private_notes','client_initiated_date','solution_code','solution']]

Converting To Date Time Format

In [4]:
df[['start_date','target_date','end_date','submitted_date','client_initiated_date','dtclose']]=df[['start_date','target_date','end_date','submitted_date','client_initiated_date','dtclose']].apply(pd.to_datetime)

In [5]:
df['start_date']=df['start_date'].dt.normalize()
df['target_date']=df['target_date'].dt.normalize()
df['submitted_date']=df['submitted_date'].dt.normalize()
df['end_date']=df['end_date'].dt.normalize()
df['dtclose']=df['dtclose'].dt.normalize()

Checking for Delay

In [6]:
x=np.busday_count(df.start_date.values.astype('datetime64[D]'), df.target_date.values.astype('datetime64[D]')+1)
y=np.busday_count(df.submitted_date.values.astype('datetime64[D]'), df.end_date.values.astype('datetime64[D]'))
z=np.busday_count(df.submitted_date.values.astype('datetime64[D]'), df.start_date.values.astype('datetime64[D]'))

In [7]:
df['planning']=x
df['delay']=y
df['VC']=z

In [8]:
df['private_notes']=df['private_notes'].fillna("")

In [9]:
client_alias_list=['Global','Local / Pack Size','COC']
subject_line=['LEAN ONE-PIECE FLOW','BH','SETUP CATEGORY / COUNTRY','AO CLEANING','CHANGE REQUEST','DELETION','UPFRONT','HARIBO D+5']
valid_words = ['GSA LATE FEEDBACK', 'COUNTRY LATE FEEDBACK', '2+ ITERATIONS( ALL FEEDBACK ON TIME )', '2+ ITERATIONS & GSA LATE FEEDBACK', '2+ ITERATIONS & COUNTRY LATE FEEDBACK', 'CODING DELAY / MISS', 'NISP ISSUE', 'VISUAL CHECK FAIL', 'VISUAL CHECK DELAY']

## Error Checking Function

In [10]:
def validate_data(df):
    error_dict = {'req_num': [], 'req_owner': [], 'error': []}

    def add_error(req_num, req_owner, error_message):
        error_dict['req_num'].append(req_num)
        error_dict['req_owner'].append(req_owner)
        error_dict['error'].append(error_message)

    for ind in df.index:
        account = df['account'][ind].lower()
        client_alias = df['client_alias'][ind]
        hours_to_response = df['hours_to_response'][ind]
        private_notes = df['private_notes'][ind]
        assets_impacted = df['assets_impacted'][ind]
        local_cat = df['local_cat'][ind]
        country_routing = df['country_routing'][ind]
        start_date = df['start_date'][ind]
        end_date = df['end_date'][ind]
        subject = df.loc[ind, 'subject']
        planning = df.loc[ind, 'planning']
        delay = df.loc[ind, 'delay']
        private_note = df.loc[ind, 'private_notes']
        subs = subject.split(' / ')[1:]

        if private_notes.strip():
            words = private_notes.split(' / ')
            for word in words:
                if word not in valid_words:
                    add_error(df['req_num'][ind], df['req_owner'][ind], 'Private note is incorrect')

        if pd.isna(df['client_initiated_date'][ind]):
            add_error(df['req_num'][ind], df['req_owner'][ind], 'Client Initiated Date is Blank')

        if 'nielsen' in account or 'tcs' in account or account == "l'oreal":
            add_error(df['req_num'][ind], df['req_owner'][ind], 'Account Name is Incorrect')

        if np.isnan(assets_impacted) or assets_impacted == 0:
            add_error(df['req_num'][ind], df['req_owner'][ind], 'Assets Impacted is either Blank or Zero')

        if pd.isna(client_alias) or client_alias not in client_alias_list:
            add_error(df['req_num'][ind], df['req_owner'][ind], 'Client Alias is either Blank or Incorrect')

        if pd.isna(local_cat):
            add_error(df['req_num'][ind], df['req_owner'][ind], 'Local Category is Blank')

        if pd.isna(country_routing):
            add_error(df['req_num'][ind], df['req_owner'][ind], 'Country Routing is Blank')

        if client_alias == 'Local / Pack Size' and not pd.isnull(hours_to_response):
            add_error(df['req_num'][ind], df['req_owner'][ind], 'For Local / Pack Size Business Hours to Response should be blank')

        if end_date < start_date:
            add_error(df['req_num'][ind], df['req_owner'][ind], 'End Date is past date than start date')
        
        
        if end_date != start_date and pd.isna(private_note):
            if delay >= 6 or planning >= 6:
                if len(subs) == 0:
                    add_error(df.loc[ind, 'req_num'], df.loc[ind, 'req_owner'], 'TAT is exceeding D+5, please update Private Note or Subject Line')
                else:
                    for sub in subs:
                        if sub not in subject_line:
                            add_error(df.loc[ind, 'req_num'], df.loc[ind, 'req_owner'], 'TAT is exceeding D+5, please update Private Note or Subject Line')

        

    return error_dict

error_dict = validate_data(df)


In [11]:
output_df=pd.DataFrame.from_dict(error_dict)
output_df=output_df.sort_values('req_owner')

## Output

In [12]:
output_df

,req_num,req_owner,error
5,REQ-04979469,Aditi Dighe,Client Alias is either Blank or Incorrect
7,REQ-04920305,Aditi Dighe,Client Alias is either Blank or Incorrect
1,REQ-04948387,Ashish Dubey,Assets Impacted is either Blank or Zero
2,REQ-04948387,Ashish Dubey,Client Alias is either Blank or Incorrect
3,REQ-04948387,Ashish Dubey,Local Category is Blank
6,REQ-04931389,Nidhi Gamit,Local Category is Blank
0,REQ-04929229,Priya Jain,Client Initiated Date is Blank
4,REQ-04958558,Ritika Nagar,Client Alias is either Blank or Incorrect
8,REQ-04926212,Shubham Saini,For Local / Pack Size Business Hours to Respon...


In [13]:
output_df.to_excel('output.xlsx', index=False)